In [1]:
import numpy as np
import import_ipynb
import pandas as pd
from data_files import elig
import central_functions as cf
from patient_visits import PATIENT_VISITS
from patients import measures2_224_001, measures2_461_012, measures2_461_014, measures2_461_018

importing Jupyter notebook from data_files.ipynb
importing Jupyter notebook from central_functions.ipynb
importing Jupyter notebook from patient_visits.ipynb


In [2]:
elig

['C:/Users/dgao/Box/Intern Data Models/224-001/224_001_elig_enroll_log.csv',
 'C:/Users/dgao/Box/Intern Data Models/224-001/elig_enrollment_log.pdf',
 'C:/Users/dgao/Box/Intern Data Models/461-005/elig.sas7bdat',
 'C:/Users/dgao/Box/Intern Data Models/461-005/eligibility.sas7bdat',
 'C:/Users/dgao/Box/Intern Data Models/461-006/elig.sas7bdat',
 'C:/Users/dgao/Box/Intern Data Models/461-006/myk461006_eligibility_20200103164652.csv',
 'C:/Users/dgao/Box/Intern Data Models/461-007/blinded_myk461007_eligibility_20200616130001.txt',
 'C:/Users/dgao/Box/Intern Data Models/461-007/elig.sas7bdat',
 'C:/Users/dgao/Box/Intern Data Models/461-008/elig.sas7bdat',
 'C:/Users/dgao/Box/Intern Data Models/461-008/myk461008_eligibility_20200604010000.txt',
 'C:/Users/dgao/Box/Intern Data Models/461-012/dta_iex_proc.txt',
 'C:/Users/dgao/Box/Intern Data Models/461-012/elig_enroll_log.pdf',
 'C:/Users/dgao/Box/Intern Data Models/461-013/elig_enroll_counts.xls',
 'C:/Users/dgao/Box/Intern Data Models/461-

These eligibility data come from the Suvoda IRT system

These data represent cardiac health measurements taken (one per patient) at the point of screening, which will typically refer to 'Screening' visits

For study_measurements table, original variables include:
Protocol: study ID MYK-###-###
Subject_Number: patient ID ###-###-###-###
Date_of_Birth: float type of year of birth
Gender: string type "Female" or "Male"
Site_ID: site ID 
QTcF_ms: an interval measurement made on an electrocardiogram used to assess some of the electrical properties of the heart; calculated in milliseconds as the time from the start of the Q wave to end of the T wave
LVEF_PCT: left ventricular ejection fraction which measures (as a percentage) how much blood is being pumped out of the left ventricle (main pumping chamber) with each contraction
Rest_LVOT_mmHG: left ventricular outflow tract, which measures cardiac systolic function and cardiac output
Valsalva_LVOT_mmHG
Post_Exercise_LVOT_mmHg
IVST_mm: interventricular septal thickness
PWT_mm
LVOT_rest
LVOT_valsalva
Maximum_Wall_Thickness
Peak_Exercise_Achieved: "Yes" or "No"
RER
O2_Saturation_PCT
(DROPPED) Age
(DROPPED) Country: IRT country

In [3]:
# OG Elig Files
# elig1_224_001 = cf.read_files(elig[0]) -- includes screening data e.g. status, exclusion reason
elig2_461_005 = cf.read_files(elig[3])
elig2_461_006 = cf.read_files(elig[5],skiprows=1,sep='|')
elig1_461_007 = cf.read_files(elig[6],skiprows=1,sep='|')
elig2_461_008 = cf.read_files(elig[9],skiprows=1,sep='|')
# elig2_461_012 = cf.read_files(elig[11])[0] -- includes screening date but no other screening data; has demographic info
# elig_461_013 = cf.read_files(elig[12]) -- ONLY aggregate counts
# elig1_461_014 = cf.read_files(elig[13]) -- inc/exc data
# elig2_461_014 = cf.read_files(elig[14],skiprows=7).iloc[:46,:] -- includes screening data e.g. status, reason for exclusion; also includes demographic data
# elig_461_015 = cf.read_files(elig[15],skiprows=8).iloc[:55,:] -- includes screening data e.g. BMI, genotype; demographic; date, comments
# elig1_461_018 = cf.read_files(elig[16]) -- inc/exc data
# elig2_461_018 = cf.read_files(elig[17]) -- excluded/completed data

In [4]:
# elig1_224_001
# elig2_461_012[0]

In [5]:
elig_461_005 = cf.read_files(elig[3])
elig_461_006 = cf.read_files(elig[5],skiprows=1,sep='|')
elig_461_007 = cf.read_files(elig[6],skiprows=1,sep='|')
elig_461_008 = cf.read_files(elig[9],skiprows=1,sep='|')

# 006: names=['study_id','patient_id','dob','sex','site_id','IVST','PWT','LVEF','LVOT_rest','LVOT_rest_intracav','LVOT_valsalva','LVOT_valsalva_intracav','LVOT_post_exer','LVOT_post_exer_intracav','QTcF','wall_thickness','age']
# 007: names=['study_id','patient_id','dob','sex','country','site_id','QTcF_rand','QTcF_w24','LVEF_w24'],
# 008: names=['study_id','patient_id','dob','sex','site_id','QTcF','QRS','age','country'],

In [6]:
def drop_cols_for_measurements(df):
    df = df.drop(['age','country','sex','dob'],errors='ignore',axis=1)
    return df

In [7]:
def rename_cols(df):
    df = df.rename(columns={'Protocol':'study_id','Subject Number':'patient_id','Subject_Number':'patient_id','Date_of_Birth':'dob','Date of Birth':'dob','Gender':'sex','Site_ID':'site_id','Site ID':'site_id','QTcF (ms) RAND':'QTcF_rand','QTcF (ms) W24':'QTcF_w24','QTcF_ms':'QTcF','QTcF (ms)':'QTcF','LVEF_PCT':'LVEF','LVEF (%)':'LVEF','LVEF (%) W24':'LVEF_w24','Rest_LVOT_mmHg':'LVOT_rest','LVOT at Rest':'LVOT_rest','LVOT at Valsalva':'LVOT_valsalva','Valsalva_LVOT_mmHg':'LVOT_valsalva','LVOT Valsalva intracavitary':'LVOT_valsalva_intracav','LVOT at Rest intracavitary':'LVOT_rest_intracav','Post_Exercise_LVOT_mmHg':'LVOT_post_exer','LVOT after Post-Exercise':'LVOT_post_exer','LVOT after Post-Exercise intracavitary':'LVOT_post_exer_intracav','IVST_mm':'IVST','PWT_mm':'PWT','Maximum_Wall_Thickness':'wall_thickness','Wall Thickness':'wall_thickness','Peak_Exercise_Achieved':'peak_exercise','O2_Saturation_PCT':'O2_sat','Age':'age','Country':'country'},errors='ignore')
    return df

In [8]:
def run_all_for_study_measures(df):
    df = rename_cols(df)
    df = drop_cols_for_measurements(df)
    df['study_id'] = df.study_id.apply(cf.format_study_id)
    df['site_id'] = df.site_id.apply(lambda x: int(x))
    return df

In [9]:
measures_461_005 = run_all_for_study_measures(elig_461_005)
measures_461_006 = run_all_for_study_measures(elig_461_006)
measures_461_007 = run_all_for_study_measures(elig_461_007)
measures_461_008 = run_all_for_study_measures(elig_461_008)

In [10]:
measures_461_006

,study_id,patient_id,site_id,IVST,PWT,LVEF,LVOT_rest,LVOT_rest_intracav,LVOT_valsalva,LVOT_valsalva_intracav,LVOT_post_exer,LVOT_post_exer_intracav,QTcF,wall_thickness
0,myk_461_006,461-006-102-605,102,0.00,7.19,77.5,6.76,0.000,4.580,0.0,13.250,0.0,478.0,15.40
1,myk_461_006,461-006-102-606,102,8.36,9.25,68.7,7.73,0.000,6.660,0.0,21.160,0.0,464.0,20.00
2,myk_461_006,461-006-102-607,102,19.21,7.59,82.4,7.18,0.000,6.860,0.0,16.480,0.0,430.0,21.80
3,myk_461_006,461-006-102-601,102,15.90,10.20,63.2,7.16,6.012,7.739,0.0,20.939,0.0,475.0,NaN
4,myk_461_006,461-006-102-602,102,10.49,6.82,72.5,5.58,0.000,2.490,0.0,8.570,0.0,466.0,18.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,myk_461_006,461-006-114-604,114,14.26,11.07,68.8,47.33,0.000,76.740,0.0,92.160,0.0,419.0,16.60
127,myk_461_006,461-006-125-609,125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,449.0,NaN
128,myk_461_006,461-006-109-611,109,13.12,9.77,70.4,6.45,0.000,7.080,0.0,14.440,0.0,479.0,20.60
129,myk_461_006,461-006-109-610,109,13.33,14.48,71.4,10.50,0.000,11.020,0.0,16.000,0.0,451.0,20.20


In [11]:
list([measures_461_005.columns,measures_461_006.columns,measures_461_007.columns,measures_461_008.columns])
study_measurements = pd.concat([measures_461_005,measures_461_006,measures_461_007,measures_461_008],axis=0).reset_index()
STUDY_MEASUREMENTS = study_measurements.rename(columns={'index':'orig_measures_index'})

In [19]:
STUDY_MEASUREMENTS

,orig_measures_index,study_id,patient_id,site_id,QTcF,LVEF,LVOT_rest,LVOT_valsalva,LVOT_post_exer,IVST,...,peak_exercise,RER,O2_sat,LVOT_rest_intracav,LVOT_valsalva_intracav,LVOT_post_exer_intracav,QTcF_rand,QTcF_w24,LVEF_w24,QRS
0,0,myk_461_005,461-005-129-502,129,477.0,69.3,129.96,102.41,174.24,NaN,...,Yes,1.03,99.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,myk_461_005,461-005-111-501,111,460.0,64.8,64.32,78.85,123.65,16.98,...,Yes,1.21,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,myk_461_005,461-005-102-501,102,451.0,77.5,50.00,0.00,55.35,19.62,...,Yes,1.11,97.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,myk_461_005,461-005-102-502,102,462.0,70.4,42.25,77.09,137.83,11.23,...,Yes,1.05,98.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,myk_461_005,461-005-132-501,132,458.0,73.6,87.24,65.29,104.86,19.86,...,Yes,1.08,98.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719,8,myk_461_008,461-008-102-113,102,456.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
720,9,myk_461_008,461-008-102-114,102,452.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
721,10,myk_461_008,461-008-103-022,103,432.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
722,11,myk_461_008,461-008-103-122,103,442.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
screen_visits = PATIENT_VISITS[(PATIENT_VISITS.reason_for_visit == 'Screening') | (PATIENT_VISITS.reason_for_visit == 'Rescreening')]

In [14]:
screen_visits.reason_for_visit.value_counts()
test = pd.merge(study_measurements,screen_visits,on=['study_id','patient_id'])
# screen_visits

In [15]:
# # Merging data across eligibility files
# elig_461_005_merge = pd.merge(left=elig1_461_005, right=elig2_461_005, how='left', left_on='patient_id', right_on='patient_id')

# elig1_461_005.head()
# elig_461_005_merge.head()